In [1]:
#To connect to ADLS G2 Here are the steps
#Create and APP registration in Active Directory
#Create a secret for the APP Registration
#Record the APP ID, the Tenant ID (for Azue Subscription) and Object ID -- these should all be visible when you create the App Registration
#Go to ADLS Gen 2 storage account
#Give that app Storage Blob Data Contributor Role (at the Storage account level not container)
#Use Storage explorer to connect to the ADLS G2 container
#Right click on the container and click manage access
#At the bottom put the APPID in the "Add User or Group" fill in box and make sure you select Read/Write Etc. Press Add
##Should work, if it doesn't... try the object ID. This part can also be coded using Azure CLI

##https://docs.databricks.com/data/data-sources/azure/azure-datalake-gen2.html



In [2]:
%scala
/*##can get the object ID with this code if needed
PS Azure:\> az ad sp show --id AppID
{
  "accountEnabled": "True",
  "addIns": [],
  "alternativeNames": [],
  "appDisplayName": "AppName",
  "appId": "AppID",
  "appOwnerTenantId": "TenantID",
  "appRoleAssignmentRequired": false,
  "appRoles": [],
  "applicationTemplateId": null,
  "deletionTimestamp": null,
  "displayName": "AppName",
  "errorUrl": null,
  "homepage": null,
  "informationalUrls": {
    "marketing": null,
    "privacy": null,
    "support": null,
    "termsOfService": null
  },
  "keyCredentials": [],
  "logoutUrl": null,
  "notificationEmailAddresses": [],
  "oauth2Permissions": [],
  "objectId": "ObjectID",
  "objectType": "ServicePrincipal",
  "odata.metadata": "https://graph.windows.net/TenantID/$metadata#directoryObjects/@Element",
  "odata.type": "Microsoft.DirectoryServices.ServicePrincipal",
  "passwordCredentials": [],
  "preferredSingleSignOnMode": null,
  "preferredTokenSigningKeyEndDateTime": null,
  "preferredTokenSigningKeyThumbprint": null,
  "publisherName": "Microsoft",
  "replyUrls": [],
  "samlMetadataUrl": null,
  "samlSingleSignOnSettings": null,
  "servicePrincipalNames": [
    "AppID"
  ],
  "servicePrincipalType": "Application",
  "signInAudience": "AzureADMyOrg",
  "tags": [
    "WindowsAzureActiveDirectoryIntegratedApp"
  ],
  "tokenEncryptionKeyId": null
}
*/

Connecting to ADLS Gen2


https://docs.databricks.com/data/data-sources/azure/azure-datalake-gen2.html

In [4]:
%scala
//This is how you comment in Scala
//"Long way" to connect to storage account and more proper
/* comment multiple lines
AppName
Application (client) ID
YOUR APPLICATIONID
(need to use Storage Explorer to give access to this objectID)
Directory (tenant) ID
YOUR TENANTID
Object ID
YOUR OBCJETID
key from blob storage
YOUR KEY FROM BLOB Storage

//object ID via power shell command 
//ecba365c-6f24-4f23-b507-88bbf0eac6c8
az ad sp show --id AppID
*/

val storageAccountName = "StorageAccountName"
val appID = "AppID"
val password = "From Secret within your App Registration"
val fileSystemName = "ContainerName"
val tenantID = "TenantID"
//AD Subscription ID - This is the TenantID

spark.conf.set("fs.azure.account.auth.type." + storageAccountName + ".dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type." + storageAccountName + ".dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id." + storageAccountName + ".dfs.core.windows.net", "" + appID + "")
spark.conf.set("fs.azure.account.oauth2.client.secret." + storageAccountName + ".dfs.core.windows.net", "" + password + "")
spark.conf.set("fs.azure.account.oauth2.client.endpoint." + storageAccountName + ".dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantID + "/oauth2/token")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://" + fileSystemName  + "@" + storageAccountName + ".dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")

Verifying that I can read files in ADLS Gen2 by putting data into a Dataframe

In [6]:
%scala

val df1 = spark
  .read
  .option ("header","True")
.option("charset", "UTF-8")
  .option ("inferschema","True")
  .csv("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Sales/dboAddresses_sales.txt")

display(df1)

AddressID,CustomerID,AddressLine1,AddressLine2,City,State,ZipCode,CreatedDate,UpdatedDate
d06fde5d-43bd-4b5c-b565-002b38303f32,1dc447c9-06e9-4741-82ba-917dd1dd8b52,982 Fifth Ave,null,Portland,OR,85101,2017-01-01T00:00:00.000+0000,2017-09-12T00:00:00.000+0000
778daea8-e9e3-4d36-bea5-0071afa39efa,e03ed1c7-4d61-4827-9a3a-451ad032c6a8,817 Cherry St,null,Austin,TX,75101,2017-01-01T00:00:00.000+0000,2017-04-07T00:00:00.000+0000
0ec3dc5b-d562-469f-959e-009560f0a197,16843ab0-5d12-49d4-b2a6-d9602d5bf750,924 Pine St,null,San Diego,CA,98101,2017-07-16T00:00:00.000+0000,2017-11-15T00:00:00.000+0000
5d13fb39-67cf-4cd6-8502-00a56d3037c7,b2f5eacf-7f6c-4009-9198-34ade9f7338e,926 Eighth Ave,null,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-01-22T00:00:00.000+0000
7e1dab8e-15f0-43d1-b099-00c4c5275272,75f6b4da-e62e-4735-97cb-00b5278b8275,445 Tenth Ave,null,New York,NY,12345,2017-01-01T00:00:00.000+0000,2017-06-03T00:00:00.000+0000
df5a8273-2500-48aa-a3e7-010e0054aa68,9a64f899-41a4-46b1-8320-f3193cae7e33,828 First Ave,null,Washington,DC,21001,2017-10-22T00:00:00.000+0000,2017-12-26T00:00:00.000+0000
e6b830be-4286-4d0a-8d90-011054ee7559,2a3b101e-4c54-47a7-b62a-3a989a869e67,921 Ninth Ave,null,Seattle,WA,92101,2017-01-01T00:00:00.000+0000,2017-08-13T00:00:00.000+0000
f99541ee-0f3b-44a8-955e-014cacd525a9,5f5ef893-9f73-4d67-889c-1d5adc42bad6,777 Ninth Ave,null,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-07-23T00:00:00.000+0000
eb70b894-a01d-45ff-8fbd-01962bbfe8fc,20a75138-e786-4c3e-a8b7-61d5f938fee7,93 Pine St,null,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-05-03T00:00:00.000+0000
4d33d894-e638-4f46-8853-01eab937c79f,5a2defe9-bb17-4c84-861b-6f69f5e479e6,884 Tenth Ave,null,Minneapolis,MN,58101,2017-01-01T00:00:00.000+0000,2017-04-03T00:00:00.000+0000


In [7]:
##Same code in Python
##Shift + enter to execute

dfpy1 = (
  spark
  .read
  .option ("header",True)
  .option ("inferschema",True)
  .csv("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Sales/dboAddresses_sales.txt")
      )

##need to redefine df1 in python... not sure why?
df1=dfpy1

## can use this command: display(df1) or
dfpy1.show()

+--------------------+--------------------+--------------+------------+-----------+-----+-------+-------------------+-------------------+
 AddressID| CustomerID| AddressLine1|AddressLine2| City|State|ZipCode| CreatedDate| UpdatedDate|
+--------------------+--------------------+--------------+------------+-----------+-----+-------+-------------------+-------------------+
d06fde5d-43bd-4b5...|1dc447c9-06e9-474...| 982 Fifth Ave| null| Portland| OR| 85101|2017-01-01 00:00:00|2017-09-12 00:00:00|
778daea8-e9e3-4d3...|e03ed1c7-4d61-482...| 817 Cherry St| null| Austin| TX| 75101|2017-01-01 00:00:00|2017-04-07 00:00:00|
0ec3dc5b-d562-469...|16843ab0-5d12-49d...| 924 Pine St| null| San Diego| CA| 98101|2017-07-16 00:00:00|2017-11-15 00:00:00|
5d13fb39-67cf-4cd...|b2f5eacf-7f6c-400...|926 Eighth Ave| null| New York| NY| 12345|2017-01-01 00:00:00|2017-01-22 00:00:00|
7e1dab8e-15f0-43d...|75f6b4da-e62e-473...| 445 Tenth Ave| null| New York| NY| 12345|2017-01-01 00:00:00|2017-06-03 00:00:00|
df5a8273-2500-48a...|9a64f899-41a4-46b...| 828 First Ave| null| Washington| DC| 21001|2017-10-22 00:00:00|2017-12-26 00:00:00|
e6b830be-4286-4d0...|2a3b101e-4c54-47a...| 921 Ninth Ave| null| Seattle| WA| 92101|2017-01-01 00:00:00|2017-08-13 00:00:00|
f99541ee-0f3b-44a...|5f5ef893-9f73-4d6...| 777 Ninth Ave| null|Minneapolis| MN| 58101|2017-01-01 00:00:00|2017-07-23 00:00:00|
eb70b894-a01d-45f...|20a75138-e786-4c3...| 93 Pine St| null|Minneapolis| MN| 58101|2017-01-01 00:00:00|2017-05-03 00:00:00|
4d33d894-e638-4f4...|5a2defe9-bb17-4c8...| 884 Tenth Ave| null|Minneapolis| MN| 58101|2017-01-01 00:00:00|2017-04-03 00:00:00|
6eb7f750-0c15-4a2...|548a6159-7964-43a...|773 Second Ave| null| Austin| TX| 75101|2017-01-01 00:00:00|2017-01-04 00:00:00|
dc0d6642-9853-4ce...|a8e763a9-529b-484...| 746 Pine St| null| Portland| OR| 85101|2017-01-01 00:00:00|2017-09-23 00:00:00|
f8a69e54-8760-4c0...|bc1f4581-a6c0-4b7...| 714 Third Ave| null| San Diego| CA| 98101|2017-01-01 00:00:00|2017-02-12 00:00:00|
75012ceb-614d-4ee...|b59f7f97-6d96-4da...| 161 Maple St| null| Portland| OR| 85101|2017-01-01 00:00:00|2017-02-19 00:00:00|
c14a1ed7-f91d-4b7...|54dd1f6a-59ea-41b...| 639 Palm St| null| Portland| OR| 85101|2017-01-01 00:00:00|2017-08-08 00:00:00|
5021d30d-4368-479...|51a1bcda-736d-4a8...| 425 Ninth Ave| null| Seattle| WA| 92101|2017-05-03 00:00:00|2017-10-21 00:00:00|
c859c3e3-ef9a-489...|0ec84e87-408a-469...| 52 Second Ave| null| Portland| OR| 85101|2017-01-01 00:00:00|2017-08-16 00:00:00|
f34de1a2-2332-46f...|6a391296-7443-485...| 124 Third Ave| null| San Diego| CA| 98101|2017-01-01 00:00:00|2017-09-21 00:00:00|
2e65faec-cb2c-49e...|233c2c7d-2c46-4a7...| 322 Third Ave| null| Austin| TX| 75101|2017-01-01 00:00:00|2017-09-02 00:00:00|
9e6afbe8-3c5f-43d...|f4601742-f4e3-47f...| 887 Sixth Ave| null| Seattle| WA| 92101|2017-01-01 00:00:00|2017-11-15 00:00:00|
+--------------------+--------------------+--------------+------------+-----------+-----+-------+-------------------+-------------------+
only showing top 20 rows

Filtering and other queries with the data using Python

In [9]:
##Column names are case sensitive and so are dataframes and so are commands
##But Dataframe names are not
filteredDF = df1.filter(df1.City == "Washington").sort(df1.ZipCode)

display(filteredDF)

##Display(filteredDF) will not work
##display(FilteredDF) will not work
##filteredDF = df1.filter(df1.city == "Washington").sort(df1.zipCode)

AddressID,CustomerID,AddressLine1,AddressLine2,City,State,ZipCode,CreatedDate,UpdatedDate
df5a8273-2500-48aa-a3e7-010e0054aa68,9a64f899-41a4-46b1-8320-f3193cae7e33,828 First Ave,null,Washington,DC,21001,2017-10-22T00:00:00.000+0000,2017-12-26T00:00:00.000+0000
476d2b97-b9d6-4adf-a20f-05506bc72550,50c91f41-778d-45b9-b3a3-aaa333012bed,458 Oak St,null,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-07-15T00:00:00.000+0000
cd435ee7-9ba9-4670-b67e-05a64fee4f5f,4a504ad5-fb1f-4b65-94d9-155ec19bb421,732 Maple St,null,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-01-19T00:00:00.000+0000
051bd2fb-cfa3-4982-8f3f-05b9d95fec84,eac60f02-55df-4626-b01c-01933fac664b,271 First Ave,null,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-01-06T00:00:00.000+0000
be94d803-aed3-420c-94ae-05c017d19d61,3d47dd8c-5063-4fce-94d5-c30e24fcedd3,432 Cherry St,null,Washington,DC,21001,2017-04-18T00:00:00.000+0000,2017-06-23T00:00:00.000+0000
2b138d92-d57d-4b5c-bfa3-05d87450cdb2,b333f9c7-9ab1-417a-a84e-3ec7cd96e124,279 Ninth Ave,null,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-11-24T00:00:00.000+0000
f0fa89f7-1a2e-4295-a0ca-07f786aa8401,8307f1c8-d9ac-471d-994b-8cc7ca0d1f98,940 Pine St,null,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-08-30T00:00:00.000+0000
ba6ae9bd-e483-47a8-87b4-0a30b0b97bbe,cfe558ba-dd6e-4d4c-8581-b3e593837935,751 First Ave,null,Washington,DC,21001,2017-03-03T00:00:00.000+0000,2017-06-17T00:00:00.000+0000
441e50ef-1cbf-4ed6-a629-0bd015e4ae26,3156657a-75dc-41a5-a5c0-3cba527509ae,111 Maple St,null,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-01-26T00:00:00.000+0000
a4e13ca7-1ebd-44f5-98f3-0f0f6acd2032,b5d8888d-2e2f-41d8-916f-88a02e905156,503 Dogwood St,null,Washington,DC,21001,2017-01-01T00:00:00.000+0000,2017-06-17T00:00:00.000+0000


In [10]:
##Aggregation
##from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import *


CityStateCount = df1.select("City","State","CustomerID")\
  .groupBy("City","State")\
  .agg(countDistinct("CustomerID").alias("Count_of_DistinctCustomers_CitiesState"))\
  .orderBy(countDistinct("CustomerID"), ascending=False)
  
#.orderBy(df1.State,ascending=False)
#.sort(countDistinct("CustomerID"))

display(CityStateCount)

City,State,Count_of_DistinctCustomers_CitiesState
Austin,TX,169
Portland,OR,162
New York,NY,158
Seattle,WA,138
San Diego,CA,133
Washington,DC,124
Minneapolis,MN,116


Printing Schema

In [12]:
CityStateCount.printSchema()

root
-- City: string (nullable = true)
-- State: string (nullable = true)
-- Count_of_DistinctCustomers_CitiesState: long (nullable = false)

Importing all of my data files into Dataframes

In [14]:
#Python Code

import pyspark

SalesOutputPath = "abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/output/dboOrders_sales.parquet"

#From Sales Database
addressdata_Sales = spark.read.format('csv').options(header='true', inferschema='true').load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Sales/dboAddresses_sales.txt")

customerdata_Sales = spark.read.format('csv').options(header='true', inferschema='true').load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Sales/dboCustomers_sales.txt")

orderdata_Sales = spark.read.format('csv').options(header='true', inferschema='true').load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Sales/dboOrders_sales.txt")

orderdetailsdata_Sales = spark.read.format('csv').options(header='true', inferschema='true').load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Sales/dboOrderDetails_sales.txt")

#Marketing Data
customerdata_Marketing = spark.read.format('csv').options(header='true', inferschema='true').load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Marketing/Customers.csv")

#Streaming Database
addressdata_Streaming= spark.read.format('csv').options(header='true', inferschema='true').load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Streaming/dboAddresses_streaming.txt")

customerdata_Streaming= spark.read.format('csv').options(header='true', inferschema='true').load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Streaming/dboCustomers_streaming.txt")

#import org.apache.spark.sql.types.StructType
from pyspark.sql.types import *
from pyspark.sql.functions import *
#Retail data
Retailschema = StructType([
    StructField("CustomerID", StringType(), True),
    StructField("FirstName", StringType(), True),
    StructField("LastName", StringType(), True),
    StructField("AddressLine1", StringType(), True),
    StructField("AddressLine2", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("ZipCode", IntegerType(), True),
    StructField("PhoneNumber", LongType(), True),
    StructField("CreatedDate", TimestampType(), True),
    StructField("UpdatedDate", TimestampType(), True)],
)
##This file does not have proper headers, so I created a schema to load the Dataframe onto with proper column names
customerdata_Retail= spark.read.format('csv').schema(Retailschema).load("abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Retail/dboCustomers.txt")
##note schema=MySchemaName

addressdata_Sales.printSchema()
customerdata_Sales.printSchema()
orderdata_Sales.printSchema()
orderdetailsdata_Sales.printSchema()
customerdata_Marketing.printSchema()
addressdata_Streaming.printSchema()
customerdata_Streaming.printSchema()
customerdata_Retail.printSchema()



root
-- AddressID: string (nullable = true)
-- CustomerID: string (nullable = true)
-- AddressLine1: string (nullable = true)
-- AddressLine2: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- ZipCode: integer (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

root
-- CustomerID: string (nullable = true)
-- LastName: string (nullable = true)
-- FirstName: string (nullable = true)
-- PhoneNumber: long (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

root
-- OrderID: string (nullable = true)
-- CustomerID: string (nullable = true)
-- OrderDate: timestamp (nullable = true)
-- ShipDate: timestamp (nullable = true)
-- TotalCost: double (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

root
-- OrderDetailID: string (nullable = true)
-- OrderID: string (nullable = true)
-- MovieID: string (nullable = true)
-- Quantity: integer (nullable = true)
-- UnitCost: double (nullable = true)
-- LineNumber: integer (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

root
-- CustomerID: string (nullable = true)
-- LastName: string (nullable = true)
-- FirstName: string (nullable = true)
-- AddressLine1: string (nullable = true)
-- AddressLine2: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- ZipCode: integer (nullable = true)
-- PhoneNumber: long (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

root
-- AddressID: string (nullable = true)
-- CustomerID: string (nullable = true)
-- AddressLine1: string (nullable = true)
-- AddressLine2: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- ZipCode: integer (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

root
-- CustomerID: string (nullable = true)
-- LastName: string (nullable = true)
-- FirstName: string (nullable = true)
-- PhoneNumber: long (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

root
-- CustomerID: string (nullable = true)
-- FirstName: string (nullable = true)
-- LastName: string (nullable = true)
-- AddressLine1: string (nullable = true)
-- AddressLine2: string (nullable = true)
-- City: string (nullable = true)
-- State: string (nullable = true)
-- ZipCode: integer (nullable = true)
-- PhoneNumber: long (nullable = true)
-- CreatedDate: timestamp (nullable = true)
-- UpdatedDate: timestamp (nullable = true)

Creating Temporary Views so I can query that data with SQL

In [16]:
##SQL Conversion time
#Create Temp views from Python

addressdata_Sales.createOrReplaceTempView("View_addressdata_Sales")
customerdata_Sales.createOrReplaceTempView("View_customerdata_Sales")

customerdata_Marketing.createOrReplaceTempView("View_customerdata_Marketing")
addressdata_Streaming.createOrReplaceTempView("View_addressdata_Streamings")
customerdata_Streaming.createOrReplaceTempView("View_customerdata_Streaming")
customerdata_Retail.createOrReplaceTempView("View_customerdata_Retail")

orderdata_Sales.createOrReplaceTempView("View_orderdata_Sales")
orderdetailsdata_Sales.createOrReplaceTempView("View_orderdetailsdata_Sales")

In [17]:
%sql
--Let's use SQL to see our data using those temp views

select cs.CustomerID,cs.FirstName, cs.LastName, cs.PhoneNumber,sa.AddressLine1,sa.AddressLine2,sa.City,sa.State,sa.ZipCode 
from View_addressdata_Sales sa
inner join View_customerdata_Sales cs on sa.CustomerID=cs.CustomerID
Union All
select customerID,FirstName, LastName, PhoneNumber,AddressLine1,AddressLine2,City,State,ZipCode
from View_customerdata_Marketing
Union ALL
select cs.CustomerID,cs.FirstName, cs.LastName, cs.PhoneNumber,ads.AddressLine1,ads.AddressLine2,ads.City,ads.State,ads.ZipCode
from View_customerdata_Streaming cs inner join
View_addressdata_Streamings ads on cs.CustomerID=ads.CustomerID
union All
select CustomerID,FirstName, LastName, PhoneNumber,AddressLine1,AddressLine2,City,State,ZipCode
from View_customerdata_Retail


/**/



CustomerID,FirstName,LastName,PhoneNumber,AddressLine1,AddressLine2,City,State,ZipCode
1dc447c9-06e9-4741-82ba-917dd1dd8b52,Trentini,Sara,5555550570,982 Fifth Ave,null,Portland,OR,85101
e03ed1c7-4d61-4827-9a3a-451ad032c6a8,Okamoto,Mizuki,5555550254,817 Cherry St,null,Austin,TX,75101
16843ab0-5d12-49d4-b2a6-d9602d5bf750,Pulido,Isidora,5555550846,924 Pine St,null,San Diego,CA,98101
b2f5eacf-7f6c-4009-9198-34ade9f7338e,Kuusik,Irene,5555550184,926 Eighth Ave,null,New York,NY,12345
75f6b4da-e62e-4735-97cb-00b5278b8275,Lukic,Gordana,5555550004,445 Tenth Ave,null,New York,NY,12345
9a64f899-41a4-46b1-8320-f3193cae7e33,Moresby,Aidan,5555550944,828 First Ave,null,Washington,DC,21001
5f5ef893-9f73-4d67-889c-1d5adc42bad6,Malladi,Dayaananda,5555550099,777 Ninth Ave,null,Minneapolis,MN,58101
20a75138-e786-4c3e-a8b7-61d5f938fee7,Codreanu,Marta,5555550356,93 Pine St,null,Minneapolis,MN,58101
5a2defe9-bb17-4c84-861b-6f69f5e479e6,Mate,Odon,5555550423,884 Tenth Ave,null,Minneapolis,MN,58101
548a6159-7964-43a6-8566-3ec29a431e67,Pelayo,Munira,5555550224,773 Second Ave,null,Austin,TX,75101


In [18]:

%sql
--create a temp view of the result set above

Create or Replace Temporary View CustomerData
as
select cs.customerID,cs.FirstName, cs.LastName, cs.PhoneNumber,sa.AddressLine1,sa.AddressLine2,sa.City,sa.State,sa.ZipCode , 'SalesDBSource' as SourceSystem
from View_addressdata_Sales sa
inner join View_customerdata_Sales cs on sa.customerid=cs.customerid
Union All
select customerID,FirstName, LastName, PhoneNumber,AddressLine1,AddressLine2,City,State,ZipCode, 'Marketing' as SourceSystem
from View_customerdata_Marketing
Union ALL
select cs.customerID,cs.FirstName, cs.LastName, cs.PhoneNumber,ads.AddressLine1,ads.AddressLine2,ads.City,ads.State,ads.ZipCode, 'Streaming' as SourceSystem
from View_customerdata_Streaming cs inner join
View_addressdata_Streamings ads on cs.customerid=ads.customerid
union All
select customerID,FirstName, LastName, PhoneNumber,AddressLine1,AddressLine2,City,State,ZipCode, 'Retail' as SourceSystem
from View_customerdata_Retail



In [19]:
%sql
select * from CustomerData

customerID,FirstName,LastName,PhoneNumber,AddressLine1,AddressLine2,City,State,ZipCode,SourceSystem
1dc447c9-06e9-4741-82ba-917dd1dd8b52,Trentini,Sara,5555550570,982 Fifth Ave,null,Portland,OR,85101,SalesDBSource
e03ed1c7-4d61-4827-9a3a-451ad032c6a8,Okamoto,Mizuki,5555550254,817 Cherry St,null,Austin,TX,75101,SalesDBSource
16843ab0-5d12-49d4-b2a6-d9602d5bf750,Pulido,Isidora,5555550846,924 Pine St,null,San Diego,CA,98101,SalesDBSource
b2f5eacf-7f6c-4009-9198-34ade9f7338e,Kuusik,Irene,5555550184,926 Eighth Ave,null,New York,NY,12345,SalesDBSource
75f6b4da-e62e-4735-97cb-00b5278b8275,Lukic,Gordana,5555550004,445 Tenth Ave,null,New York,NY,12345,SalesDBSource
9a64f899-41a4-46b1-8320-f3193cae7e33,Moresby,Aidan,5555550944,828 First Ave,null,Washington,DC,21001,SalesDBSource
5f5ef893-9f73-4d67-889c-1d5adc42bad6,Malladi,Dayaananda,5555550099,777 Ninth Ave,null,Minneapolis,MN,58101,SalesDBSource
20a75138-e786-4c3e-a8b7-61d5f938fee7,Codreanu,Marta,5555550356,93 Pine St,null,Minneapolis,MN,58101,SalesDBSource
5a2defe9-bb17-4c84-861b-6f69f5e479e6,Mate,Odon,5555550423,884 Tenth Ave,null,Minneapolis,MN,58101,SalesDBSource
548a6159-7964-43a6-8566-3ec29a431e67,Pelayo,Munira,5555550224,773 Second Ave,null,Austin,TX,75101,SalesDBSource


In [20]:
%sql

Create or Replace Temporary View SalesData
as
select os.orderID,CustomerID,date(OrderDate) as OrderDate,date(ShipDate) as ShipDate,TotalCost,MovieID,Quantity, UnitCost, LineNumber
from View_orderdata_Sales os inner join 
View_orderdetailsdata_Sales ods on os.orderid=ods.orderid




In [21]:
%sql
--Now, let's use SQL to see the combined number of customers from multiple data sets and how many live in each city/state Combination
-- and we will write it as a CTE!
WITH CityData
as
(
select City, State, Count(*) as CountOfCustomers
from CustomerData
Group by City, State
)
Select *
from CityData
Order by CountOfCustomers desc



City,State,CountOfCustomers
Austin,TX,340
Portland,OR,337
New York,NY,300
Seattle,WA,287
San Diego,CA,257
Washington,DC,244
Minneapolis,MN,234


In [22]:
%sql
--Map it by state
select State, Count(*) as CountOfCustomers
from CustomerData
Group by State
--

State,CountOfCustomers
MN,234
DC,244
OR,337
CA,257
WA,287
NY,300
TX,340


Inserting Summarized data into my ADLS Gen2 Data Lake

In [24]:
###insert summarized data back into datalake

CustomerDataPath = "abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Output/CustomerData.parquet"
SalesDataPath = "abfss://ContainerName@StorageAccountName.dfs.core.windows.net/Demo/Output/SalesData.parquet"

#this is great code below - works the same as the one above but does not need to have a dataframe created first

spark.sql("select * from CustomerData").write.option("compression","snappy").mode("overwrite").parquet(CustomerDataPath)
spark.sql("select * from SalesData").write.option("compression","snappy").mode("overwrite").parquet(SalesDataPath)


##similar functionality, different coding preference
#dfcustomerdata = spark.sql("select cs.FirstName, cs.LastName, cs.PhoneNumber,sa.AddressLine1,sa.AddressLine2,sa.City,sa.State,sa.ZipCode from View_addressdata_Sales sa inner join View_customerdata_Sales cs on sa.customerid=cs.customerid")

#.write

#(dfcustomerdata.write 
#    .option("compression","snappy")
#    .parquet(SalesOutputPath)
#)

Writing my Data to Azure SQL Database/Azure SQL Managed Instance



https://docs.microsoft.com/en-us/azure/sql-database/sql-database-spark-connector

In [26]:
%scala

/*
For documentation purposes only
These are the objects I created in my SQL Database

CREATE LOGIN databricks
	WITH PASSWORD = 'password!' 
GO

Create Database Databrickslanding
Go

Use Databrickslanding
GO

CREATE USER databricks
	FOR LOGIN databricks
GO

-- Add user to the database owner role
EXEC sp_addrolemember N'db_datareader', N'databricks'
GO

EXEC sp_addrolemember N'db_datawriter', N'databricks'
GO

Create Table CustomerDataSum
(
City varchar(100),
State varchar(3),
CountOfCustomers int
)


Create Table CustomerData
(
customerID varchar(100),
FirstName varchar(100),
LastName varchar(100),
PhoneNumber varchar(15),
AddressLine1 varchar(100),
AddressLine2 varchar(100),
City varchar(100),
State varchar(3),
ZipCode varchar(9),
SourceSystem varchar (100)
)

Create Table SalesData
(
orderID varchar(100),
customerID varchar(100),
OrderDate datetime,
ShiptDate datetime,
TotalCost Float,
MovieID varchar(100),
Quantity int,
UnitCost Float,
LineNumber int
)

create procedure [dbo].[usp_truncateData]
as

truncate table dbo.CustomerDataSum
Truncate Table CustomerData
Truncate Table SalesData

GO

GRANT EXECUTE on [dbo].[usp_truncateData] to Databricks
GRANT ALTER ON [dbo].[CustomerData] TO [databricks]
GRANT ALTER ON [dbo].[CustomerDataSum] TO [databricks]
GRANT ALTER ON [dbo].[SalesData] TO [databricks]
*/

In [27]:
%scala

Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")

//Regular Azure SQL
//val jdbcHostname = "AzureSQLServerName.database.windows.net"
//val jdbcPort = 1433
//val jdbcDatabase = "databrickslanding"

//Azure SQL MI
val jdbcHostname = "SQLMIServerName.public.UNIQUEADDRESS.database.windows.net"
val jdbcPort = 3342
val jdbcDatabase = "databrickslanding"



// Create the JDBC URL without passing in the user and password parameters.
val jdbcUrl = s"jdbc:sqlserver://${jdbcHostname}:${jdbcPort};database=${jdbcDatabase}"

// Create a Properties() object to hold the parameters.
import java.util.Properties
val connectionProperties = new Properties()

connectionProperties.put("user", s"databricks")
connectionProperties.put("password", s"password!")

val driverClass = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
connectionProperties.setProperty("Driver", driverClass)

val dfCustomerDataSumSQL = spark.read.jdbc(jdbcUrl, "CustomerDataSum", connectionProperties)
//Spark automatically reads the schema from the database table and maps its types back to Spark SQL types.


dfCustomerDataSumSQL.printSchema

display(dfCustomerDataSumSQL)


City,State,CountOfCustomers


In [28]:
%sql

Create or Replace Temporary View CustomerSumDataTemp
as
select City,State, Count(*) as CountOfCustomers
from CustomerData
Group by City,State

In [29]:
%scala

spark.table("CustomerSumDataTemp")
     .write
     .mode(SaveMode.Append) // <--- Overwrite the existing table neds special permissions (Create Table - because it drops and recreates)
     .jdbc(jdbcUrl, "CustomerDataSum", connectionProperties)

In [30]:
%scala
display(dfCustomerDataSumSQL)

City,State,CountOfCustomers
Minneapolis,MN,234
Washington,DC,244
Austin,TX,340
New York,NY,300
San Diego,CA,257
Portland,OR,337
Seattle,WA,287


Another way to do it with Delta Lake and a Databricks "Database"

Some cleanup first

In [33]:
%sql 
Create Database if NOT Exists AymanDB

In [34]:
%sql
drop table if exists aymanDB.customerdata_Streaming;
drop table if exists aymanDB.addressdata_Streaming;

drop table if exists aymanDB.customerdata_Sales;
drop table if exists aymanDB.addressdata_Sales;

drop table if exists aymanDB.customerdata_Marketing;

drop table if exists aymanDB.customerdata_Retail;

drop table if exists aymanDB.customers_Final;

Creating a path in DBFS (Databricks File System)

In [36]:
dbutils.fs.rm('/DeltaTest/',True)

Out[119]: True

In [37]:
dbutils.fs.mkdirs('/DeltaTest/')

Out[120]: True

Using the same Dataframes created above, we can write the same data to DELTA tables

In [39]:
##Insert data into Delta Lake Tables
customerdata_Sales.write.format("delta").save("dbfs:/DeltaTest/silver/customerdata_Sales")
spark.sql("CREATE TABLE aymanDB.customerdata_Sales USING DELTA LOCATION 'dbfs:/DeltaTest/silver/customerdata_Sales'")

addressdata_Sales.write.format("delta").save("dbfs:/DeltaTest/silver/addressdata_Sales")
spark.sql("CREATE TABLE aymanDB.addressdata_Sales USING DELTA LOCATION 'dbfs:/DeltaTest/silver/addressdata_Sales'")

customerdata_Streaming.write.format("delta").save("dbfs:/DeltaTest/silver/customerdata_Streaming")
spark.sql("CREATE TABLE aymanDB.customerdata_Streaming USING DELTA LOCATION 'dbfs:/DeltaTest/silver/customerdata_Streaming'")

addressdata_Streaming.write.format("delta").save("dbfs:/DeltaTest/silver/addressdata_Streaming")
spark.sql("CREATE TABLE aymanDB.addressdata_Streaming USING DELTA LOCATION 'dbfs:/DeltaTest/silver/addressdata_Streaming'")

customerdata_Marketing.write.format("delta").save("dbfs:/DeltaTest/silver/customerdata_Marketing")
spark.sql("CREATE TABLE aymanDB.customerdata_Marketing USING DELTA LOCATION 'dbfs:/DeltaTest/silver/customerdata_Marketing'")

customerdata_Retail.write.format("delta").save("dbfs:/DeltaTest/silver/customerdata_Retail")
spark.sql("CREATE TABLE aymanDB.customerdata_Retail USING DELTA LOCATION 'dbfs:/DeltaTest/silver/customerdata_Retail'")



Out[121]: DataFrame[]

Create Gold Delta Lake Table

In [41]:
%sql
create table if not exists aymandb.customers_Final
(
CustomerID string,
FirstName string,
LastName string,
AddressLine1 string,
AddressLine2 string,
City string,
State String,
Zipcode String,
PhoneNumber String,
UpdatedDate timestamp,
SourceSystem string
) 
using delta
OPTIONS (path 'dbfs:/DeltaTest/gold/customers_Final')
partitioned by (CustomerID)

In [42]:
%sql
select *
from aymandb.customers_Final
limit 20

CustomerID,FirstName,LastName,AddressLine1,AddressLine2,City,State,Zipcode,PhoneNumber,UpdatedDate,SourceSystem


Take all the data and put it in the Gold table

In [44]:
%sql
Insert into aymanDB.customers_Final
select customerID,FirstName, LastName, AddressLine1,AddressLine2,City,State,ZipCode,PhoneNumber, current_date() as UpdatedDate, 'Marketing' as Source
from aymanDB.customerdata_Marketing
union All
select CustomerID,FirstName, LastName, AddressLine1,AddressLine2,City,State,ZipCode, PhoneNumber,current_date() as UpdatedDate, 'Retail' as Source
from aymanDB.customerdata_Retail
UNION ALL
select cs.CustomerID,cs.FirstName, cs.LastName, cs.PhoneNumber,sa.AddressLine1,sa.AddressLine2,sa.City,sa.State,sa.ZipCode,current_date() as UpdatedDate, 'Sales' as Source
from aymanDB.addressdata_Sales sa
inner join aymanDB.customerdata_Sales cs on sa.CustomerID=cs.CustomerID
Union ALL
select cs.CustomerID,cs.FirstName, cs.LastName, cs.PhoneNumber,ads.AddressLine1,ads.AddressLine2,ads.City,ads.State,ads.ZipCode,current_date() as UpdatedDate, 'Streaming' as Source
from aymanDB.customerdata_Streaming cs inner join
aymanDB.addressdata_Streaming ads on cs.CustomerID=ads.CustomerID


In [45]:
%sql
select *
from aymandb.customers_Final
limit 20



CustomerID,FirstName,LastName,AddressLine1,AddressLine2,City,State,Zipcode,PhoneNumber,UpdatedDate,SourceSystem
87D5880B-0030-4580-AE85-432CE3F17263,Buscan,Elisabeta,254 Cherry St,NULL,Minneapolis,MN,58101,5555559429,2020-05-21T00:00:00.000+0000,Marketing
8AD7BE72-568A-44B7-A512-1A472D0BE270,Angelovaa,Nadezhda,88 Cherry St,NULL,Washington,DC,21001,5555550540,2020-05-21T00:00:00.000+0000,Marketing
4B302B91-A1D1-4C7A-BC98-8BBB37339124,Petrzelka,Lukas,167 Dogwood St,NULL,Portland,OR,85101,5555559255,2020-05-21T00:00:00.000+0000,Marketing
3FC5A08A-F8EB-43E3-BBD1-9723DE2C17E4,Jovanovic,Dragan,171 Third Ave,NULL,New York,NY,12345,5555559576,2020-05-21T00:00:00.000+0000,Marketing
C02CA2BF-06C2-454C-8BEB-B9867388716C,Sonti,Bhagavati,841 Ninth Ave,NULL,New York,NY,12345,5555559200,2020-05-21T00:00:00.000+0000,Marketing
75C820EA-C108-4278-B527-78F59726CE00,Deyoung,Clifton,628 Fifth Ave,NULL,New York,NY,12345,5555550526,2020-05-21T00:00:00.000+0000,Marketing
7d195360-408a-48ed-ac78-86cabd9a17bb,Agne,Lukauskiene,5555550516,185 Seventh Ave,null,Washington,DC,21001,2020-05-21T00:00:00.000+0000,Streaming
314D398A-551E-46C8-80DF-CD600FC1B760,Benson,Sophie,489 Fifth Ave,NULL,Portland,OR,85101,5555559756,2020-05-21T00:00:00.000+0000,Marketing
48d444b7-9bef-417c-89c1-688802379252,Earnestine,Wiggins,5555550388,397 Ninth Ave,null,San Diego,CA,98101,2020-05-21T00:00:00.000+0000,Streaming
9dfe5db7-b117-4ace-ab1b-f31fac607b41,Beniamin,Marandici,5555550945,607 Tenth Ave,null,Seattle,WA,92101,2020-05-21T00:00:00.000+0000,Streaming


In [46]:
%sql
select SourceSystem, count(SourceSystem) as CountBySourceSystem
from aymandb.customers_Final
group by SourceSystem

SourceSystem,CountBySourceSystem
Sales,999
Marketing,250
Streaming,500
Retail,250
